In [1]:
#Import packages
#---------------------------------------
import sys
import os
import glob
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import pygtftk as gtftk
import pyranges as pr

#Import your modules
#---------------------------------------
import te_rna_f as te
import admin_tools as ad

# Define paths
#----------------------------------------------------------------------
l_code = '/Users/dominicburrows/Dropbox/PhD/Analysis/my_scripts/GitHub/'
l_data = '/Users/dominicburrows/Dropbox/PhD/analysis/Project/'
l_fig = '/Users/dominicburrows/Dropbox/PhD/figures/'

s_code = '/cndd3/dburrows/CODE/'
s_data = '/cndd3/dburrows/DATA/'
s_fig = '/cndd3/dburrows/FIGS/'

%load_ext autoreload
sys.version

'3.9.12 (main, Apr  5 2022, 06:56:58) \n[GCC 7.5.0]'

In [2]:
#Read in required files for filtering
bed_pl = pd.read_csv('/cndd3/dburrows/DATA/te/gtf/bed/rmsk.hg38.filt-5ptrim.sort.plus.bed', sep='\t', header=None)
bed_pl.columns =['Chromosome', 'Start', 'End', 'Strand', 'transcript_id', 'gene_id', 'family_id', 'class_id']
bed_mi = pd.read_csv('/cndd3/dburrows/DATA/te/gtf/bed/rmsk.hg38.filt-5ptrim.sort.minus.bed',sep='\t', header=None)
bed_mi.columns =['Chromosome', 'Start', 'End', 'Strand', 'transcript_id', 'gene_id', 'family_id', 'class_id']

bam_pl = pr.read_bam('/cndd3/dburrows/DATA/te/rna/filt.prac/prac/prac.plus.filt.bam', as_df=True)
bam_mi = pr.read_bam('/cndd3/dburrows/DATA/te/rna/filt.prac/prac/prac.minus.filt.bam', as_df=True)

#File checks
assert sum(bam_pl['Strand'] == '+') == len(bam_pl), 'Some non plus strands assigned to plus bam'
assert sum(bam_mi['Strand'] == '-') == len(bam_mi), 'Some non minus strands assigned to minus bam'
assert sum(bed_pl['Strand'] == '+') == len(bed_pl), 'Some non plus strands assigned to plus bed'
assert sum(bed_mi['Strand'] == '-') == len(bed_mi), 'Some non minus strands assigned to minus bed'


In [3]:
#DEF for +/-
# Filter out reads that do not overlap with 5' portion of insertion

curr_bed = bed_pl
curr_bam = bam_pl
curr_name = pd.read_csv('/cndd3/dburrows/DATA/te/rna/filt.prac/prac/prac.plus.umi.txt', sep='\t', header=None)
assert len(curr_bam) == len(curr_name), 'Bam and UMI files not the same length'
curr_bam['UMI']=curr_name[0].values

#Loop through each chromosome
chr_unq = np.unique(curr_bam['Chromosome'].values)
for i,chr in enumerate(chr_unq):
    print(i, chr)

0 chr1
1 chr10
2 chr11
3 chr12
4 chr13
5 chr14
6 chr15
7 chr16
8 chr17
9 chr18
10 chr19
11 chr2
12 chr20
13 chr21
14 chr22
15 chr3
16 chr4
17 chr5
18 chr6
19 chr7
20 chr8
21 chr9
22 chrX
23 chrY


In [4]:
#Slice bed/bam files by chromosome
chr = chr_unq[0]
chr_bam = curr_bam[curr_bam['Chromosome'] == chr]
chr_bed = curr_bed[curr_bed['Chromosome'] == chr]



In [5]:
#Generate flattened vector of 5' insertion positions and their indeces
all_bed_pos = np.asarray([(np.arange(chr_bed['Start'].values[i], chr_bed['End'].values[i]+1), np.full(chr_bed['End'].values[i]+1 - chr_bed['Start'].values[i],i)) for i in range(len(chr_bed))]) #list of all bed positions for each insertion and their indeces
assert len(all_bed_pos) == len(chr_bed), 'Not all bed positions accounted for'

flat_bed_pos = np.ravel(np.asarray(all_bed_pos)[:,0,:]) # flattened vector of all 5' regions across all insertions
flat_bed_ind = np.ravel(np.asarray(all_bed_pos)[:,1,:]) # flattened vector of indeces for each region that maps it back onto the original bed file
assert len(flat_bed_pos) == len(flat_bed_ind), 'Bed position and index vectors not the same length'

#Check that all remaining reads align to bed file 
#assert sum(np.in1d(curr_bam['Start'].values, flat_bed_pos)) == len(curr_bam), 'Not all reads align to bed file'

In [6]:
pr = np.asarray([np.arange(chr_bam['Start'].values[i], chr_bam['End'].values[i]+1) for i in range(len(chr_bam))])


/tmp/ipykernel_1724654/2096402078.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pr = np.asarray([np.arange(chr_bam['Start'].values[i], chr_bam['End'].values[i]+1) for i in range(len(chr_bam))])


In [8]:
count

276195

In [20]:
# get BAM file of final aligning reads
bam_bool = np.in1d(chr_bam['Start'].values, flat_bed_pos) #Boolean of indeces of reads whose tss overlaps with bed files
bam_final = chr_bam[bam_bool] #Final bam reads that have aligned to 5' ends

#Assert??? -> check they all indeed align?? 
bam_final

,Chromosome,Start,End,Strand,Flag,UMI
36,chr1,90923,91011,+,355,A00132:417:HKVTWDSX3:3:1325:11315:2738
47,chr1,101055,101153,+,99,A00132:417:HKVTWDSX3:4:2422:10936:2080
60,chr1,124578,124641,+,99,A00132:417:HKVTWDSX3:4:2240:25238:7059
245,chr1,141671,141767,+,99,A00132:417:HKVTWDSX3:3:1456:16640:17065
246,chr1,141684,141784,+,99,A00132:417:HKVTWDSX3:4:1210:9390:14622
...,...,...,...,...,...,...
355253,chr1,248761660,248761737,+,419,A00132:417:HKVTWDSX3:3:1138:21423:20431
355354,chr1,248841652,248841749,+,163,A00132:417:HKVTWDSX3:3:2155:32371:35164
355355,chr1,248841652,248841749,+,163,A00132:417:HKVTWDSX3:3:2155:32199:35368
355356,chr1,248841652,248841749,+,163,A00132:417:HKVTWDSX3:3:2203:10854:18474


In [21]:
test = np.asarray([np.arange(bam_final['Start'].values[i], bam_final['Start'].values[i]+1) for i in range(len(bam_final))])


In [24]:
start = time.time()
count=0
for t in test:
    if np.any(np.in1d(t, flat_bed_pos)):
        count+=1
print(time.time()-start)

339.8513836860657


In [25]:
count

14567

In [26]:
len(test)

14567

In [3]:
%%bash
str=$1

#Define paths and variables
inpath=/cndd3/dburrows/DATA/te/rna/PE.fastq/trimmed
outpath=/cndd3/dburrows/DATA/te/rna/PE.bam/
id_arr=($(find $inpath/ -maxdepth 1 -name "*Sample*$str*"))

#Loop over each sample
for i in ${id_arr[@]}
do
    echo Running $i 
    r1=$i/R1-merge_val_1.fq.gz
    r2=$i/R2-merge_val_2.fq.gz
    
    mkdir $outpath/$(basename $i)
    cd $outpath/$(basename $i)

    STAR --genomeDir /cndd3/dburrows/DATA/te/gtf//annotations/gencode/STAR.gencode.v37 --readFilesIn $r1 $r2 --outSAMunmapped None --outFilterType BySJout --outSAMattributes All --outFilterMultimapNmax 100 --outFilterMismatchNmax 999 --outFilterMismatchNoverReadLmax 0.04 --alignIntronMin 20 --alignIntronMax 1000000 --alignMatesGapMax 1000000 --alignSJoverhangMin 8 --alignSJDBoverhangMin 1 --sjdbScore 1 --readFilesCommand zcat --runThreadN 16 --genomeLoad LoadAndKeep --limitBAMsortRAM 10000000000 --outSAMtype BAM SortedByCoordinate --quantMode TranscriptomeSAM GeneCounts --winAnchorMultimapNmax 200 --outMultimapperOrder Random --outSAMmultNmax -1 
    samtools index Aligned.sortedByCoord.out.bam
    cp $CODE3/te_ageing/workspace.sh $outpath/$(basename $i)/log.workspace
done





/cndd3/dburrows/DATA/te/rna/PE.fastq/trimmed/Sample_HCTYPA008-GLU/R1-merge_val_1.fq.gz /cndd3/dburrows/DATA/te/rna/PE.fastq/trimmed/Sample_HCTYPA008-GLU/R2-merge_val_2.fq.gz
/cndd3/dburrows/DATA/te/rna/PE.fastq/trimmed/Sample_5086-GLU/R1-merge_val_1.fq.gz /cndd3/dburrows/DATA/te/rna/PE.fastq/trimmed/Sample_5086-GLU/R2-merge_val_2.fq.gz
/cndd3/dburrows/DATA/te/rna/PE.fastq/trimmed/Sample_4379-GABA/R1-merge_val_1.fq.gz /cndd3/dburrows/DATA/te/rna/PE.fastq/trimmed/Sample_4379-GABA/R2-merge_val_2.fq.gz
/cndd3/dburrows/DATA/te/rna/PE.fastq/trimmed/Sample_4332-GABA/R1-merge_val_1.fq.gz /cndd3/dburrows/DATA/te/rna/PE.fastq/trimmed/Sample_4332-GABA/R2-merge_val_2.fq.gz
/cndd3/dburrows/DATA/te/rna/PE.fastq/trimmed/Sample_1848-GABA/R1-merge_val_1.fq.gz /cndd3/dburrows/DATA/te/rna/PE.fastq/trimmed/Sample_1848-GABA/R2-merge_val_2.fq.gz
/cndd3/dburrows/DATA/te/rna/PE.fastq/trimmed/Sample_Hct17HEIA010-GABA/R1-merge_val_1.fq.gz /cndd3/dburrows/DATA/te/rna/PE.fastq/trimmed/Sample_Hct17HEIA010-GABA/R2-

In [62]:
#Check that all files are present
import glob
import os

full_l = os.listdir('/cndd2/Public_Datasets/Dracheva_PsychEncode_development/raw_May2022/') 
os.chdir('/cndd3/dburrows/DATA/te/rna/PE.fastq/trimmed/')
check_l = glob.glob('*Samp*')

import numpy as np
not_int = np.setxor1d(check_l, full_l)
assert len(full_l) == len(check_l)
assert len(not_int) == 0

In [ ]:
#Save first 4 columns
samtools view Aligned.sortedByCoord.out.bam |cut -f 1,2,3,4 > prac.txt

In [ ]:
#Filter

samtools view -h Aligned.sortedByCoord.out.bam | awk 'NR==FNR{a[$0];next} !($1"\t"$2"\t"$3"\t"$4 in a)' prac-head.txt -> out.prac